In [ ]:
import rtsvg
rt = rtsvg.RACETrack()
import ollama
from pydantic import BaseModel
import time
import os

class NumberClueOrientation(BaseModel):
    number:      int
    clue:        str
    orientation: str

class CrosswordPuzzleEntries(BaseModel):
    items: list[NumberClueOrientation]

model    = 'qwen2.5vl:32b' # 'llama4:scout' # 'qwen2.5vl:7b' # 'gemma3:27b' # 'qwen2.5vl:32b' # 'gemma3:12b' 'llama3.2-vision:11b' #
client   = ollama.Client()
_prompt_ = """Extract the crossword entries."""

_lu_           = {}
_process_time_ = {}

_screenshot_dir_ = '../../../data/crossword_puzzle_screenshots/'
_files_ = os.listdir(_screenshot_dir_)

In [ ]:
if False:
    for _file_ in _files_:
        screenshots = []
        if _file_.endswith('.jpeg'): 
            screenshots.append(os.path.join(_screenshot_dir_, _file_))
            print(screenshots)
            t0 = time.time()
            response = client.chat(model=model, messages=[
                {
                    'role':    'user',
                    'content': _prompt_,
                    'images':  screenshots,
                },],
                format=CrosswordPuzzleEntries.model_json_schema(),
                options={'num_ctx': 8192+2048}
            )
            t1 = time.time()
            _items_ = CrosswordPuzzleEntries.model_validate_json(response['message']['content'])
            _lu_[_file_], _process_time_[_file_] = _items_, t1-t0

In [ ]:
#
# Group By number and orientation -- and into a set to see where there are duplicates (or discrepancies)
#
entry_to_clues = {}
entry_to_files = {}
for x in _lu_:
    for y in _lu_[x].items:
        entry = (y.number, y.orientation.lower())
        if entry not in entry_to_clues: 
            entry_to_clues[entry] = set()
            entry_to_files[entry] = set()
        entry_to_clues[entry].add(y.clue)
        entry_to_files[entry].add(x)

In [ ]:
#
# Just dump this to a file and fix by hand
# ... first part is dump / second part is load
#

#_tuples_ = list(entry_to_clues.keys())
#print('{')
#for x in sorted(_tuples_, key=lambda x: (x[1], x[0])): print(x, ':', entry_to_clues[x], ',')
#print('}')

for _file_ in _files_:
    if _file_.endswith('_entries.txt') and _file_.startswith('.') == False:
        print(_file_)
        with open(os.path.join(_screenshot_dir_, _file_), 'rt') as f:
            crossword_entries = eval(f.read())

In [ ]:
#
# The following produced something that may be usable...
# Model:  'qwen2.5vl:32b'
#
_prompt_ = """Extract out the numbered cells.  Return as a JSON object that includes the number, the x index, and the y index.
"""
'''
for _file_ in _files_:
    screenshots = []
    if _file_.endswith('.jpeg'): 
        screenshots.append(os.path.join(_screenshot_dir_, _file_))
        print(screenshots)
        t0 = time.time()
        response = client.chat(model=model, messages=[
            {
                'role':    'user',
                'content': _prompt_,
                'images':  screenshots,
            },],
            options={'num_ctx': 8192+2048}
        )
        t1 = time.time()
        print(response['message']['content'])
        break
'''

In [ ]:
# Results from 'qwen2.5vl:32b' ... it's pixel level and not grid level...
# ... and it's not actually from the crossword puzzle screenshot... it's just a grid of numbers :(
_results_ = [
    {"number": 1, "x": 304, "y": 104},{"number": 2, "x": 354, "y": 104},{"number": 3, "x": 404, "y": 104},{"number": 4, "x": 454, "y": 104},
    {"number": 5, "x": 504, "y": 104},{"number": 6, "x": 554, "y": 104},{"number": 7, "x": 604, "y": 104},{"number": 8, "x": 654, "y": 104},
    {"number": 9, "x": 704, "y": 104},{"number": 10, "x": 754, "y": 104},{"number": 11, "x": 304, "y": 154},{"number": 12, "x": 354, "y": 154},
    {"number": 13, "x": 404, "y": 154},{"number": 14, "x": 454, "y": 154},{"number": 15, "x": 504, "y": 154},{"number": 16, "x": 554, "y": 154},
    {"number": 17, "x": 604, "y": 154},{"number": 18, "x": 654, "y": 154},{"number": 19, "x": 704, "y": 154},
    {"number": 20, "x": 754, "y": 154},{"number": 21, "x": 304, "y": 204},{"number": 22, "x": 354, "y": 204},
    {"number": 23, "x": 404, "y": 204},{"number": 24, "x": 454, "y": 204},{"number": 25, "x": 504, "y": 204},
    {"number": 26, "x": 554, "y": 204},{"number": 27, "x": 604, "y": 204},{"number": 28, "x": 654, "y": 204},
    {"number": 29, "x": 704, "y": 204},{"number": 30, "x": 754, "y": 204},{"number": 31, "x": 304, "y": 254},{"number": 32, "x": 354, "y": 254},
    {"number": 33, "x": 404, "y": 254},{"number": 34, "x": 454, "y": 254},{"number": 35, "x": 504, "y": 254},{"number": 36, "x": 554, "y": 254},
    {"number": 37, "x": 604, "y": 254},{"number": 38, "x": 654, "y": 254},{"number": 39, "x": 704, "y": 254},{"number": 40, "x": 754, "y": 254},
    {"number": 41, "x": 304, "y": 304},{"number": 42, "x": 354, "y": 304},{"number": 43, "x": 404, "y": 304},{"number": 44, "x": 454, "y": 304},
    {"number": 45, "x": 504, "y": 304},{"number": 46, "x": 554, "y": 304},{"number": 47, "x": 604, "y": 304},{"number": 48, "x": 654, "y": 304},
    {"number": 49, "x": 704, "y": 304},
    {"number": 50, "x": 754, "y": 304},
    {"number": 51, "x": 304, "y": 354},
    {"number": 52, "x": 354, "y": 354},
    {"number": 53, "x": 404, "y": 354},
    {"number": 54, "x": 454, "y": 354},
    {"number": 55, "x": 504, "y": 354},
    {"number": 56, "x": 554, "y": 354},
    {"number": 57, "x": 604, "y": 354},
    {"number": 58, "x": 654, "y": 354},
    {"number": 59, "x": 704, "y": 354},
    {"number": 60, "x": 754, "y": 354},
    {"number": 61, "x": 304, "y": 404},
    {"number": 62, "x": 354, "y": 404},
    {"number": 63, "x": 404, "y": 404},
    {"number": 64, "x": 454, "y": 404},
    {"number": 65, "x": 504, "y": 404},
    {"number": 66, "x": 554, "y": 404},
    {"number": 67, "x": 604, "y": 404},
    {"number": 68, "x": 654, "y": 404},
    {"number": 69, "x": 704, "y": 404},
    {"number": 70, "x": 754, "y": 404},
    {"number": 71, "x": 304, "y": 454},
    {"number": 72, "x": 354, "y": 454},
    {"number": 73, "x": 404, "y": 454},
    {"number": 74, "x": 454, "y": 454},
    {"number": 75, "x": 504, "y": 454},
    {"number": 76, "x": 554, "y": 454},
    {"number": 77, "x": 604, "y": 454},
    {"number": 78, "x": 654, "y": 454},
    {"number": 79, "x": 704, "y": 454},
    {"number": 80, "x": 754, "y": 454},
    {"number": 81, "x": 304, "y": 504},
    {"number": 82, "x": 354, "y": 504},
    {"number": 83, "x": 404, "y": 504},
    {"number": 84, "x": 454, "y": 504},
    {"number": 85, "x": 504, "y": 504},
    {"number": 86, "x": 554, "y": 504},
    {"number": 87, "x": 604, "y": 504},
    {"number": 88, "x": 654, "y": 504},
    {"number": 89, "x": 704, "y": 504},
    {"number": 90, "x": 754, "y": 504},
    {"number": 91, "x": 304, "y": 554},
    {"number": 92, "x": 354, "y": 554},
    {"number": 93, "x": 404, "y": 554},
    {"number": 94, "x": 454, "y": 554},
    {"number": 95, "x": 504, "y": 554},
    {"number": 96, "x": 554, "y": 554},
    {"number": 97, "x": 604, "y": 554},
    {"number": 98, "x": 654, "y": 554},
    {"number": 99, "x": 704, "y": 554},
    {"number": 100, "x": 754, "y": 554},
    {"number": 101, "x": 304, "y": 604},
    {"number": 102, "x": 354, "y": 604},
    {"number": 103, "x": 404, "y": 604},
    {"number": 104, "x": 454, "y": 604},
    {"number": 105, "x": 504, "y": 604},
    {"number": 106, "x": 554, "y": 604},
    {"number": 107, "x": 604, "y": 604},
    {"number": 108, "x": 654, "y": 604},
    {"number": 109, "x": 704, "y": 604},
    {"number": 110, "x": 754, "y": 604},
    {"number": 111, "x": 304, "y": 654},
    {"number": 112, "x": 354, "y": 654},
    {"number": 113, "x": 404, "y": 654},
    {"number": 114, "x": 454, "y": 654},
    {"number": 115, "x": 504, "y": 654},
    {"number": 116, "x": 554, "y": 654},
    {"number": 117, "x": 604, "y": 654},
    {"number": 118, "x": 654, "y": 654},
    {"number": 119, "x": 704, "y": 654},
    {"number": 120, "x": 754, "y": 654},
    {"number": 121, "x": 304, "y": 704}
]
svg = [f'<svg x="0" y="0" width="800" height="800"> <rect x="0" y="0" width="800" height="800" x="0" y="0" fill="#ffffff" />']
x0, y0, x1, y1 = 1e9, 1e9, -1e9, -1e9
xs, ys = {}, {}
for _d_ in _results_:
    _number_, _x_, _y_ = _d_['number'], _d_['x'], _d_['y']
    if _x_ not in xs: xs[_x_] = 0
    if _y_ not in ys: ys[_y_] = 0
    xs[_x_] += 1
    ys[_y_] += 1
    x0, y0, x1, y1 = min(x0, _x_), min(y0, _y_), max(x1, _x_), max(y1, _y_)
    svg.append(f'<circle cx="{_x_}" cy="{_y_}" r="3" fill="none" stroke="#000000" stroke-width="0.1" />')
    svg.append(rt.svgText(str(_number_), _x_, _y_, txt_h=10.0, color='#000000', anchor='middle'))
for _x_ in xs.keys(): svg.append(f'<line x1="{_x_}" y1="{y0}" x2="{_x_}" y2="{y1}" stroke="black" stroke-width="1" />')
for _y_ in ys.keys(): svg.append(f'<line x1="{x0}" y1="{_y_}" x2="{x1}" y2="{_y_}" stroke="black" stroke-width="1" />')
svg.append('</svg>')
rt.tile([''.join(svg)])